# Домашнее задание

Data contains the book rating information. Ratings (Book-Rating) are either explicit, expressed on a scale from 1-10 (higher values denoting higher appreciation), or implicit, expressed by 0.

### 1. Реализовать персональный топ  - принимает на вход возраст и локацию, на выходе персональный топ   - 1 балл

Персональный топ - это топ товаров по похожим возрасту/интересам/локации. Как сделать? Разбить на сегменты по выбраным признакам. Топ делать по книгам с хорошим средним рейтингом.

### 2. На основе метода кластеризации похожих пользователей построить рекомендации (Слайд 27) - 3 балла

Нужно топ-10 рекомендаций с самой высокой оценкой. Считаем среднюю оценку для каждой книги по кластеру и выводим топ-10 книг.

### 3. Совстречаемость - 3 балла

В совстречаемости также учитывать оценки. Вес пары книг встретившихся у пользователя - полусумма их оценок.

### 4. Коллаборативная фильтрация - 3 балла

Коллаборативную фильтрацию реализовывать как на слайде 51 презентации, посоветовав каждому пользователю топ-10 книг с самой высокой оценкой. Сделать рекомендации User-based и Item-based и сравнить.

Если совсем сложно - можно сделать как в семинарской части, поставив оценку "0", если рейтинг < 5 и "1" - в противном случае. Тогда максимум за это - 1 балл. Реализовать U2I и I2I рекомендации.

### Примечание:

Так как пользователей много - можно зафиксировать несколько произвольных и для них уже составлять рекомендации

Работоспособность I2I можно проверять на известных книгах (Гарри Поттер, Властелин Колец, Интервью с вампиром, Код-Да-Винчи, Маленький Принц)

Рейтинг книг обязательно нужно учитывать

Не забываем также предобработать данные - выкинуть выбросы-пользователей и выбросы-книги.

# Жесткий дедлайн 20 мая 23:59

Выводить в качестве рекомендаций лучше названия книг, картинки (если они есть) и соответствующие метрики близости.

In [265]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from PIL import Image
from io import BytesIO
import requests
from scipy.sparse import coo_matrix, csr_matrix


In [266]:
books = pd.read_csv("BX-Books.csv.gz")

/tmp/ipykernel_96384/263975756.py:1: DtypeWarning: Columns (3) have mixed types. Specify dtype option on import or set low_memory=False.
  books = pd.read_csv("BX-Books.csv.gz")


In [267]:
interactions = pd.read_csv("BX-Book-Ratings.csv.gz", sep=";", encoding = "ISO-8859-1")
interactions = interactions[interactions["Book-Rating"] != 0]

In [268]:
interactions

,User-ID,ISBN,Book-Rating
1,276726,0155061224,5
3,276729,052165615X,3
4,276729,0521795028,6
6,276736,3257224281,8
7,276737,0600570967,6
...,...,...,...
1149773,276704,0806917695,5
1149775,276704,1563526298,9
1149777,276709,0515107662,10
1149778,276721,0590442449,10


In [269]:
books

,ISBN,Book-Title,Book-Author,Year-Of-Publication,Publisher,Image-URL-S,Image-URL-M,Image-URL-L
0,0195153448,Classical Mythology,Mark P. O. Morford,2002,Oxford University Press,http://images.amazon.com/images/P/0195153448.0...,http://images.amazon.com/images/P/0195153448.0...,http://images.amazon.com/images/P/0195153448.0...
1,0002005018,Clara Callan,Richard Bruce Wright,2001,HarperFlamingo Canada,http://images.amazon.com/images/P/0002005018.0...,http://images.amazon.com/images/P/0002005018.0...,http://images.amazon.com/images/P/0002005018.0...
2,0060973129,Decision in Normandy,Carlo D'Este,1991,HarperPerennial,http://images.amazon.com/images/P/0060973129.0...,http://images.amazon.com/images/P/0060973129.0...,http://images.amazon.com/images/P/0060973129.0...
3,0374157065,Flu: The Story of the Great Influenza Pandemic...,Gina Bari Kolata,1999,Farrar Straus Giroux,http://images.amazon.com/images/P/0374157065.0...,http://images.amazon.com/images/P/0374157065.0...,http://images.amazon.com/images/P/0374157065.0...
4,0393045218,The Mummies of Urumchi,E. J. W. Barber,1999,W. W. Norton &amp; Company,http://images.amazon.com/images/P/0393045218.0...,http://images.amazon.com/images/P/0393045218.0...,http://images.amazon.com/images/P/0393045218.0...
...,...,...,...,...,...,...,...,...
271355,0440400988,There's a Bat in Bunk Five,Paula Danziger,1988,Random House Childrens Pub (Mm),http://images.amazon.com/images/P/0440400988.0...,http://images.amazon.com/images/P/0440400988.0...,http://images.amazon.com/images/P/0440400988.0...
271356,0525447644,From One to One Hundred,Teri Sloat,1991,Dutton Books,http://images.amazon.com/images/P/0525447644.0...,http://images.amazon.com/images/P/0525447644.0...,http://images.amazon.com/images/P/0525447644.0...
271357,006008667X,Lily Dale : The True Story of the Town that Ta...,Christine Wicker,2004,HarperSanFrancisco,http://images.amazon.com/images/P/006008667X.0...,http://images.amazon.com/images/P/006008667X.0...,http://images.amazon.com/images/P/006008667X.0...
271358,0192126040,Republic (World's Classics),Plato,1996,Oxford University Press,http://images.amazon.com/images/P/0192126040.0...,http://images.amazon.com/images/P/0192126040.0...,http://images.amazon.com/images/P/0192126040.0...


In [270]:
books_meets = interactions.groupby("ISBN")["User-ID"].count().reset_index().rename(columns={"User-ID": "user_num"})

In [271]:
user_meets = interactions.groupby("User-ID")["ISBN"].count().reset_index().rename(columns={"ISBN": "books_num"})

In [272]:
interactions = interactions.merge(books_meets, on=["ISBN"]).merge(user_meets, on=["User-ID"])

In [273]:
interactions

,User-ID,ISBN,Book-Rating,user_num,books_num
0,276726,0155061224,5,1,1
1,276729,052165615X,3,1,2
2,276729,0521795028,6,1,2
3,276736,3257224281,8,4,1
4,86583,3257224281,6,4,18
...,...,...,...,...,...
433666,276664,0631189629,9,1,4
433667,276664,0747205051,7,1,4
433668,276668,3423120754,8,1,1
433669,276679,8804413530,8,1,1


In [274]:
interactions = interactions[(interactions["user_num"] > 5) & 
                            (interactions["books_num"] > 5) &
                            (interactions["books_num"] < 200)]

In [275]:
interactions

,User-ID,ISBN,Book-Rating,user_num,books_num
5,86583,3404139178,9,8,18
6,86583,3453061187,8,13,18
13,86583,3426609061,7,8,18
8555,16877,038550120X,9,81,10
8556,16877,034539657X,7,26,10
...,...,...,...,...,...
412270,108487,1573220183,10,6,11
412271,108487,1571312471,10,6,11
412736,250227,0380795272,8,8,9
420027,205839,0312971842,8,8,6


In [276]:
users = pd.read_csv('BX-Users.csv.gz', delimiter=';', encoding = 'ISO-8859-1')

In [277]:
users

,User-ID,Location,Age
0,1,"nyc, new york, usa",NaN
1,2,"stockton, california, usa",18.0
2,3,"moscow, yukon territory, russia",NaN
3,4,"porto, v.n.gaia, portugal",17.0
4,5,"farnborough, hants, united kingdom",NaN
...,...,...,...
278853,278854,"portland, oregon, usa",NaN
278854,278855,"tacoma, washington, united kingdom",50.0
278855,278856,"brampton, ontario, canada",NaN
278856,278857,"knoxville, tennessee, usa",NaN


In [278]:
interactions = interactions.merge(books[["ISBN", "Image-URL-M", "Book-Title"]].rename(
    columns={"Image-URL-M": "picture_url"}), on=["ISBN"])

In [279]:
interactions = interactions.merge(users, on=["User-ID"])

In [280]:
# Не понял мува в мердже урлов. Лучше смерджить пользователей по возрастам
interactions

,User-ID,ISBN,Book-Rating,user_num,books_num,picture_url,Book-Title,Location,Age
0,86583,3404139178,9,8,18,http://images.amazon.com/images/P/3404139178.0...,Das Lacheln der Fortuna: Historischer Roman,"stuttgart, , germany",31.0
1,86583,3453061187,8,13,18,http://images.amazon.com/images/P/3453061187.0...,Die Jury. Roman.,"stuttgart, , germany",31.0
2,86583,3426609061,7,8,18,http://images.amazon.com/images/P/3426609061.0...,House Of God.,"stuttgart, , germany",31.0
3,132500,3404139178,10,8,43,http://images.amazon.com/images/P/3404139178.0...,Das Lacheln der Fortuna: Historischer Roman,"fürth, ,",29.0
4,132500,3499232871,8,11,43,http://images.amazon.com/images/P/3499232871.0...,Herzsprung,"fürth, ,",29.0
...,...,...,...,...,...,...,...,...,...
110514,24172,0590483404,8,9,20,http://images.amazon.com/images/P/0590483404.0...,"A Shocker on Shock Street (Goosebumps, No 35)","gawler sth, south australia, australia",45.0
110515,190082,0525460543,7,6,6,http://images.amazon.com/images/P/0525460543.0...,The Prince of Egypt Collector's Edition Storybook,"henderson, kentucky, usa",42.0
110516,206652,1860465811,9,8,10,http://images.amazon.com/images/P/1860465811.0...,Wind-Up Bird Chronicle,"edinburgh, scotland, united kingdom",37.0
110517,223525,1588720284,8,6,13,http://images.amazon.com/images/P/1588720284.0...,Why Your Life Sucks: And What You Can Do Abou...,"san diego, california, usa",NaN


In [281]:
from sklearn import preprocessing
from sklearn.feature_extraction.text import CountVectorizer
le = preprocessing.LabelEncoder()

In [282]:
interactions["product_id"] = le.fit_transform(interactions["ISBN"])
interactions["vid"] = le.fit_transform(interactions["User-ID"])

In [283]:
interactions

,User-ID,ISBN,Book-Rating,user_num,books_num,picture_url,Book-Title,Location,Age,product_id,vid
0,86583,3404139178,9,8,18,http://images.amazon.com/images/P/3404139178.0...,Das Lacheln der Fortuna: Historischer Roman,"stuttgart, , germany",31.0,10397,3445
1,86583,3453061187,8,13,18,http://images.amazon.com/images/P/3453061187.0...,Die Jury. Roman.,"stuttgart, , germany",31.0,10495,3445
2,86583,3426609061,7,8,18,http://images.amazon.com/images/P/3426609061.0...,House Of God.,"stuttgart, , germany",31.0,10428,3445
3,132500,3404139178,10,8,43,http://images.amazon.com/images/P/3404139178.0...,Das Lacheln der Fortuna: Historischer Roman,"fürth, ,",29.0,10397,5248
4,132500,3499232871,8,11,43,http://images.amazon.com/images/P/3499232871.0...,Herzsprung,"fürth, ,",29.0,10552,5248
...,...,...,...,...,...,...,...,...,...,...,...
110514,24172,0590483404,8,9,20,http://images.amazon.com/images/P/0590483404.0...,"A Shocker on Shock Street (Goosebumps, No 35)","gawler sth, south australia, australia",45.0,7079,879
110515,190082,0525460543,7,6,6,http://images.amazon.com/images/P/0525460543.0...,The Prince of Egypt Collector's Edition Storybook,"henderson, kentucky, usa",42.0,6211,7498
110516,206652,1860465811,9,8,10,http://images.amazon.com/images/P/1860465811.0...,Wind-Up Bird Chronicle,"edinburgh, scotland, united kingdom",37.0,10209,8112
110517,223525,1588720284,8,6,13,http://images.amazon.com/images/P/1588720284.0...,Why Your Life Sucks: And What You Can Do Abou...,"san diego, california, usa",NaN,10138,8754


In [284]:
csr_rates = coo_matrix((interactions["Book-Rating"], (interactions["vid"], interactions["product_id"])), 
                            shape=(len(set(interactions["vid"])), len(set(interactions["product_id"]))))

In [285]:
def chunks(l, n):
    for i in range(0, len(l), n):
        yield l[i:i + n]

def rec_imaging(product_ids, content_dict, measure=None, top_n=5):
    picture_urls = [content_dict[i] for i in product_ids]
    cnt = 0
    for idx, chunk in enumerate(chunks(picture_urls, top_n)):
        fig = plt.figure(figsize=(20, 4))
        for n, i in enumerate(chunk):
            try:
                r = requests.get(i)
                im = Image.open(BytesIO(r.content))

            except:
                print('Something went wrong with {url}'.format(url=i))
                continue

            a = fig.add_subplot(1, top_n, n + 1)
            if measure is not None:
                a.title.set_text("measure = {}".format(np.round(measure[cnt], 4)))
                cnt += 1
            plt.imshow(im)
            plt.axis('off')
        plt.show()

class Top():
    def __init__(self, Int):
        # Создаём словарь для того, чтобы по product_id возвращать url товара
        self.product_id_to_url = {}
        for i in Int[["product_id", "picture_url"]].drop_duplicates().values:
            self.product_id_to_url[i[0]] = i[1]
        self.interactions = Int
        
    def top_items(self):
        # Хотим получить в отсортированном порядке датафрейм - product_id, user_number
        items = self.interactions\
                                                .groupby("product_id")["vid"].count() \
                                                .reset_index() \
                                                .sort_values("vid", ascending=False)[:30]
        
        rec_imaging(items["product_id"].values, self.product_id_to_url, items["vid"].values)

### Ищем id нужных книг

In [286]:
for i, j in interactions[["product_id", "Book-Title"]].drop_duplicates().values:
    if "Tree" in j:
        print("idx:", i, "\tBook Title:", j)

idx: 7319 	Book Title: The Mulberry Tree
idx: 294 	Book Title: The Bean Trees
idx: 1255 	Book Title: The Baron in the Trees
idx: 4622 	Book Title: Cold Sassy Tree
idx: 9362 	Book Title: The Education of Little Tree (A Zia Book)
idx: 8145 	Book Title: Knight At Dawn (Magic Tree House 2, paper)
idx: 3246 	Book Title: Cold Sassy Tree
idx: 8633 	Book Title: The Mulberry Tree
idx: 606 	Book Title: The Bean Trees
idx: 182 	Book Title: The Giving Tree
idx: 9861 	Book Title: Courage Tree
idx: 336 	Book Title: A Tree Grows in Brooklyn
idx: 3449 	Book Title: The Lexus and the Olive Tree: Understanding Globalization
idx: 1374 	Book Title: Far from the Tree
idx: 4359 	Book Title: Cold Sassy Tree
idx: 181 	Book Title: The Giving Tree
idx: 8144 	Book Title: Dinosaurs Before Dark  (Magic Tree House #1)
idx: 9232 	Book Title: Shade of the Tree
idx: 8389 	Book Title: Silver on the Tree
idx: 659 	Book Title: The Legacy of Luna: The Story of a Tree, a Woman and the Struggle to Save the Redwoods
idx: 279 

In [287]:
interactions[(interactions["Age"] == 28)]

,User-ID,ISBN,Book-Rating,user_num,books_num,picture_url,Book-Title,Location,Age,product_id,vid
2543,184532,0743211383,6,57,105,http://images.amazon.com/images/P/0743211383.0...,Dreamcatcher,"des moines, iowa, usa",28.0,8477,7290
2544,184532,059035342X,10,313,105,http://images.amazon.com/images/P/059035342X.0...,Harry Potter and the Sorcerer's Stone (Harry P...,"des moines, iowa, usa",28.0,7036,7290
2545,184532,0451167805,7,30,105,http://images.amazon.com/images/P/0451167805.0...,Firestarter,"des moines, iowa, usa",28.0,5420,7290
2546,184532,0451180232,8,47,105,http://images.amazon.com/images/P/0451180232.0...,Nightmares &amp; Dreamscapes,"des moines, iowa, usa",28.0,5459,7290
2547,184532,0439139597,10,137,105,http://images.amazon.com/images/P/0439139597.0...,Harry Potter and the Goblet of Fire (Book 4),"des moines, iowa, usa",28.0,4226,7290
...,...,...,...,...,...,...,...,...,...,...,...
110276,17172,0552998931,6,14,8,http://images.amazon.com/images/P/0552998931.0...,Chocolat,"móstoles, madrid, spain",28.0,6270,609
110304,276039,8845915611,7,6,11,http://images.amazon.com/images/P/8845915611.0...,Le Ceneri Di Angela,"città di castello, umbria, italy",28.0,10704,10853
110352,59967,0140023747,7,6,27,http://images.amazon.com/images/P/0140023747.0...,Puckoon,"dundee, scotland, united kingdom",28.0,810,2375
110397,127164,0440223458,5,7,8,http://images.amazon.com/images/P/0440223458.0...,A Dry Spell,"south lebanon, ohio, usa",28.0,4439,5040


### 1. Реализовать персональный топ  - принимает на вход возраст и локацию, на выходе персональный топ   - 1 балл

Персональный топ - это топ товаров по похожим возрасту/интересам/локации. Как сделать? Разбить на сегменты по выбраным признакам. Топ делать по книгам с хорошим средним рейтингом.

In [398]:
def get_personal_top(age, location = None):
    filtered_interactions = interactions[(interactions["Age"] == age)]
    if location:
        filtered_interactions = filtered_interactions[(filtered_interactions["Location"] == location)]

    book_ratings = filtered_interactions.groupby("product_id")["Book-Rating"].mean().reset_index()
    personal_top = book_ratings.nlargest(5, "Book-Rating")

    personal_top_with_titles = personal_top.merge(interactions[["product_id", "Book-Title", "picture_url"]], on="product_id", how="left").drop_duplicates().reset_index()

    return personal_top_with_titles


In [289]:
# Не знаю почему, но картинки не отображаются
t = Top(interactions)
# t.top_items()

# r = requests.get(interactions["picture_url"][0])
# im = Image.open(BytesIO(r.content))
# plt.imshow(im)

In [387]:
interactions.groupby("Location")["Location"].count().nlargest(10)

Location
toronto, ontario, canada          1292
chicago, illinois, usa            1154
portland, oregon, usa              959
austin, texas, usa                 695
san diego, california, usa         684
seattle, washington, usa           647
houston, texas, usa                637
ottawa, ontario, canada            593
calgary, alberta, canada           570
san francisco, california, usa     558
Name: Location, dtype: int64

In [399]:
age = 19
location = "nyc, new york, usa"
location = "toronto, ontario, canada"
personal_top = get_personal_top(age, location)
personal_top[["Book-Title", "Book-Rating"]]

,Book-Title,Book-Rating
0,The Screwtape Letters,10.0
1,Like Water for Chocolate : A Novel in Monthly ...,10.0
2,The Tragedy of Macbeth: With New and Updated C...,10.0
3,Hearts In Atlantis,10.0
4,The Virgin Suicides,9.0


In [291]:
get_personal_top(age)

,product_id,Book-Rating,Book-Title,picture_url
0,44,10.0,Elements of Style 3ED,http://images.amazon.com/images/P/0024182001.0...
5,45,10.0,Politically Correct Bedtime Stories: Modern Ta...,http://images.amazon.com/images/P/002542730X.0...
58,47,10.0,Once upon a More Enlightened Time: More Politi...,http://images.amazon.com/images/P/0028604199.0...
84,140,10.0,Bel Canto,http://images.amazon.com/images/P/0060188731.0...
108,167,10.0,Moment of Truth,http://images.amazon.com/images/P/0060196092.0...


### 2. На основе метода кластеризации похожих пользователей построить рекомендации (Слайд 27) - 3 балла

Нужно топ-10 рекомендаций с самой высокой оценкой. Считаем среднюю оценку для каждой книги по кластеру и выводим топ-10 книг.

In [315]:
from sklearn.preprocessing import LabelEncoder, StandardScaler
from sklearn.cluster import KMeans

def get_recommendations(interactions):
    # Заполнение пропущенных значений в поле "Age"
    interactions["Age"].fillna(interactions["Age"].mean(), inplace=True)
    
    # Предобработка данных
    label_encoder = LabelEncoder()
    encoded_location = label_encoder.fit_transform(interactions["Location"])
    
    scaler = StandardScaler()
    scaled_age = scaler.fit_transform(interactions[["Age"]])
    
    features = np.concatenate((scaled_age, encoded_location.reshape(-1, 1)), axis=1)
    
    # Кластеризация пользователей
    kmeans = KMeans(n_clusters=5)  # Задайте желаемое количество кластеров
    cluster_labels = kmeans.fit_predict(features)
    
    # Расчет средней оценки для каждой книги внутри каждого кластера
    interactions["Cluster"] = cluster_labels
    book_ratings = interactions.groupby(["Cluster", "product_id"])["Book-Rating"].mean().reset_index()

    # Получение топ-10 рекомендаций
    top_recommendations = book_ratings.groupby("product_id")["Book-Rating"].mean().nlargest(10).reset_index()
    
    return top_recommendations.merge(interactions[["product_id", "Book-Title"]], on="product_id", how="left").drop_duplicates().reset_index()


In [316]:
recommended_books = get_recommendations(interactions)
recommended_books_with_info = recommended_books.merge(interactions, on="product_id", how="left")

/home/timofey/.local/lib/python3.10/site-packages/sklearn/cluster/_kmeans.py:870: FutureWarning: The default value of `n_init` will change from 10 to 'auto' in 1.4. Set the value of `n_init` explicitly to suppress the warning
  warnings.warn(


In [318]:
recommended_books

,index,product_id,Book-Rating,Book-Title
0,0,42,10.0,Mere Christianity: A revised and enlarged edit...
1,4,49,10.0,"The Lord of the rings,"
2,8,342,10.0,First They Killed My Father : A Daughter of Ca...
3,9,363,10.0,A People's History of the United States: 1492 ...
4,11,683,10.0,Mrs. Piggle-Wiggle's Magic
5,13,808,10.0,The Razor's Edge
6,15,971,10.0,Before I Say Goodbye
7,17,1022,10.0,The House at Pooh Corner
8,19,1026,10.0,Black Beauty (Puffin Classics)
9,20,1110,10.0,Moby Dick (Penguin Popular Classics)


In [341]:
from sklearn.preprocessing import LabelEncoder, StandardScaler
from sklearn.cluster import KMeans

def train_cluster_model(interactions, n_clasters=5):
    # Заполнение пропущенных значений в поле "Age"
    interactions["Age"].fillna(interactions["Age"].mean(), inplace=True)
    
    # Предобработка данных
    label_encoder = LabelEncoder()
    encoded_location = label_encoder.fit_transform(interactions["Location"])
    
    scaler = StandardScaler()
    scaled_age = scaler.fit_transform(interactions[["Age"]])
    
    features = np.concatenate((scaled_age, encoded_location.reshape(-1, 1)), axis=1)
    
    # Кластеризация пользователей
    kmeans = KMeans(n_clusters=n_clasters)
    cluster_labels = kmeans.fit_predict(features)
    
    # Добавление меток кластеров в исходный датасет
    interactions["Cluster"] = cluster_labels
    
    return kmeans, label_encoder, scaler


def get_personal_top_recommendations(interactions, user_id, kmeans_model, label_encoder, scaler):
    # Находим кластер, к которому принадлежит целевой пользователь
    target_user_cluster = interactions[interactions["vid"] == user_id]["Cluster"].values[0]
    
    # Получаем всех пользователей в том же кластере
    cluster_users = interactions[interactions["Cluster"] == target_user_cluster]
    
    # Расчет средней оценки для каждой книги только внутри кластера целевого пользователя
    book_ratings = cluster_users.groupby("product_id")["Book-Rating"].mean().reset_index()
    
    # Если книга находится в списке взаимодействий пользователя внутри кластера, то оценка книги сохраняется, в противном случае оценка устанавливается в 0
    book_ratings["Metric"] = book_ratings.apply(lambda row: row["Book-Rating"] if row["product_id"] in cluster_users[cluster_users["vid"] == user_id]["product_id"].values else 0, axis=1)
    
    # Получение топ-10 рекомендаций для целевого пользователя
    top_recommendations = book_ratings.nlargest(10, "Metric").merge(interactions[["Book-Title", "product_id"]], on="product_id").drop_duplicates()
    
    return top_recommendations




In [424]:
kmeans_model, label_encoder, scaler = train_cluster_model(interactions, n_clasters=15)

/home/timofey/.local/lib/python3.10/site-packages/sklearn/cluster/_kmeans.py:870: FutureWarning: The default value of `n_init` will change from 10 to 'auto' in 1.4. Set the value of `n_init` explicitly to suppress the warning
  warnings.warn(


In [346]:
personal_top_recommendations = get_personal_top_recommendations(interactions, 3445, kmeans_model, label_encoder, scaler)
personal_top_recommendations

,product_id,Book-Rating,Metric,Book-Title
0,10397,9.0,9.0,Das Lacheln der Fortuna: Historischer Roman
6,10495,8.0,8.0,Die Jury. Roman.
13,10428,7.0,7.0,House Of God.
19,3,8.0,0.0,Angelas Ashes
21,6,6.5,0.0,Angelas Ashes
46,15,8.0,0.0,Unless: A Novel
51,16,6.0,0.0,Unless : A Novel
59,19,5.0,0.0,Sniglets (Snig'lit : Any Word That Doesn't App...
65,21,7.0,0.0,Joshua
74,22,10.0,0.0,GREAT GATSBY (REISSUE)


In [419]:
get_personal_top_recommendations(interactions, 3450, kmeans_model, label_encoder, scaler)


,product_id,Book-Rating,Metric,Book-Title
0,7984,9.000000,9.000000,The Woman Warrior : Memoirs of a Girlhood Amon...
17,1941,8.500000,8.500000,Bitter Medicine
25,2802,8.000000,8.000000,Touch
28,6696,8.000000,8.000000,The Ugly Little Boy
32,8063,7.666667,7.666667,The English Patient
90,6,10.000000,0.000000,Angelas Ashes
115,9,6.000000,0.000000,Red Leaves
123,13,10.000000,0.000000,Billy
134,18,7.500000,0.000000,The Bride Stripped Bare
138,20,6.500000,0.000000,The GREAT GATSBY (A Scribner Classic)


In [423]:
get_personal_top_recommendations(interactions, 4500, kmeans_model, label_encoder, scaler)

,product_id,Book-Rating,Metric,Book-Title
0,10565,10.000000,10.0,"Endlich Nichtleser: Die beste Methode, mit dem..."
5,10511,8.000000,8.0,Ruf. Mich. An.
8,10586,8.000000,8.0,Schande
15,10462,6.500000,6.5,Dunkle Kammern.
19,0,7.333333,0.0,Clara Callan
25,1,10.000000,0.0,The Forgetting Room: A Fiction (Byzantium Book)
30,2,7.000000,0.0,Miss Wyoming Uk Edition
34,4,5.000000,0.0,Lost Girls
40,5,7.000000,0.0,Before and After
44,6,7.333333,0.0,Angelas Ashes


### 3. Совстречаемость - 3 балла

В совстречаемости также учитывать оценки. Вес пары книг встретившихся у пользователя - полусумма их оценок.


In [412]:
interactions.drop(["User-ID", "ISBN", "Cluster"], axis=1)

,Book-Rating,user_num,books_num,picture_url,Book-Title,Location,Age,product_id,vid
0,9,8,18,http://images.amazon.com/images/P/3404139178.0...,Das Lacheln der Fortuna: Historischer Roman,"stuttgart, , germany",31.000000,10397,3445
1,8,13,18,http://images.amazon.com/images/P/3453061187.0...,Die Jury. Roman.,"stuttgart, , germany",31.000000,10495,3445
2,7,8,18,http://images.amazon.com/images/P/3426609061.0...,House Of God.,"stuttgart, , germany",31.000000,10428,3445
3,10,8,43,http://images.amazon.com/images/P/3404139178.0...,Das Lacheln der Fortuna: Historischer Roman,"fürth, ,",29.000000,10397,5248
4,8,11,43,http://images.amazon.com/images/P/3499232871.0...,Herzsprung,"fürth, ,",29.000000,10552,5248
...,...,...,...,...,...,...,...,...,...
110514,8,9,20,http://images.amazon.com/images/P/0590483404.0...,"A Shocker on Shock Street (Goosebumps, No 35)","gawler sth, south australia, australia",45.000000,7079,879
110515,7,6,6,http://images.amazon.com/images/P/0525460543.0...,The Prince of Egypt Collector's Edition Storybook,"henderson, kentucky, usa",42.000000,6211,7498
110516,9,8,10,http://images.amazon.com/images/P/1860465811.0...,Wind-Up Bird Chronicle,"edinburgh, scotland, united kingdom",37.000000,10209,8112
110517,8,6,13,http://images.amazon.com/images/P/1588720284.0...,Why Your Life Sucks: And What You Can Do Abou...,"san diego, california, usa",36.016799,10138,8754


In [413]:
def get_recommendations_based_on_cooccurrence(interactions, user_id):
    # Фильтрация взаимодействий только для целевого пользователя
    user_interactions = interactions[interactions["vid"] == user_id]
    
    # Группировка взаимодействий по книгам и расчет суммы оценок для каждой пары книг
    book_cooccurrence = user_interactions.groupby(["product_id", "Book-Title"])["Book-Rating"].sum().reset_index()
    
    # Расчет средней оценки для каждой книги отдельно
    book_ratings = interactions.groupby(["product_id", "Book-Title"])["Book-Rating"].mean().reset_index()
    
    # Объединение данных о совстречаемости и оценках книг
    recommendations = book_cooccurrence.merge(book_ratings, on=["product_id", "Book-Title"], how="left")
    
    # Расчет метрики на основе полусуммы оценок пары книг
    recommendations["Metric"] = recommendations["Book-Rating_x"] / 2 + recommendations["Book-Rating_y"] / 2
    
    # Сортировка рекомендаций по убыванию метрики
    recommendations = recommendations.sort_values("Metric", ascending=False)
    
    return recommendations


In [416]:
user_id = 3445
recommendations = get_recommendations_based_on_cooccurrence(interactions, user_id)
recommendations

,product_id,Book-Title,Book-Rating_x,Book-Rating_y,Metric
0,10397,Das Lacheln der Fortuna: Historischer Roman,9,8.833333,8.916667
2,10495,Die Jury. Roman.,8,7.857143,7.928571
1,10428,House Of God.,7,5.333333,6.166667


In [418]:
get_recommendations_based_on_cooccurrence(interactions, 3450)

,product_id,Book-Title,Book-Rating_x,Book-Rating_y,Metric
3,7984,The Woman Warrior : Memoirs of a Girlhood Amon...,9,7.764706,8.382353
0,1941,Bitter Medicine,8,8.500000,8.250000
2,6696,The Ugly Little Boy,8,8.000000,8.000000
1,2802,Touch,8,6.666667,7.333333
4,8063,The English Patient,7,7.379310,7.189655


In [420]:
get_recommendations_based_on_cooccurrence(interactions, 4500)

,product_id,Book-Title,Book-Rating_x,Book-Rating_y,Metric
2,10565,"Endlich Nichtleser: Die beste Methode, mit dem...",10,9.200000,9.600000
3,10586,Schande,8,8.428571,8.214286
1,10511,Ruf. Mich. An.,8,6.666667,7.333333
0,10462,Dunkle Kammern.,5,6.250000,5.625000
